In [14]:
from prompts import TOOL_DESC
from tools import check_weather, multiply, list_files, allowed_location_dir

list_tool = [
    check_weather, 
    multiply, 
    list_files, 
    allowed_location_dir
]

[print(t.args_schema.model_json_schema()) for t in list_tool]


{'properties': {'location': {'description': 'The location to check the weather for.', 'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'WeatherInput', 'type': 'object'}
{'properties': {'a': {'description': 'The first number.', 'title': 'A', 'type': 'number'}, 'b': {'description': 'The second number.', 'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'MultiplyInput', 'type': 'object'}
{'properties': {'directory': {'default': None, 'description': 'Directory path to list files from. Leave empty to use the default allowed path.', 'title': 'Directory', 'type': 'string'}}, 'title': 'ListFilesInput', 'type': 'object'}
{'description': 'Return the absolute path of a predefined directory that the agent can access.', 'properties': {}, 'title': 'allowed_location_dir', 'type': 'object'}


[None, None, None, None]

In [38]:
from pydantic import BaseModel

def generate_tool_prompt(name_for_model: str, 
                        name_for_human: str, 
                        description_for_model: str, 
                        schema: type[BaseModel]) -> str:
    # Get the JSON Schema (Pydantic v2)
    schema_dict = schema.model_json_schema()

    # Extract and format parameter descriptions
    param_descriptions = []
    properties = schema_dict.get("properties", None)
    if properties:
        for field_name, field_info in properties.items():
            desc = field_info.get("description", "No description")
            type_ = field_info.get("type", "unknown")
            required = field_name in schema_dict.get("required", [])
            required_str = "required" if required else "optional"
            param_descriptions.append(f"- `{field_name}` ({type_}, {required_str}): {desc}")
    else: # has no properties
        param_descriptions = ["- This tool takes no parameters."]
    parameters = "\n".join(param_descriptions)

    # Final prompt
    prompt = (
        f"{name_for_model}: Call this tool to interact with the {name_for_human} API.\n"
        f"What is the {name_for_human} API useful for? {description_for_model}\n"
        f"Parameters:\n{parameters}\n\n"
    )

    return prompt

In [39]:
tool_desc = ""
for t in list_tool:
    tool_desc+=generate_tool_prompt(name_for_model=t.name,
                        name_for_human=t.name,
                        description_for_model=t.description,
                        schema=t.args_schema)
print(tool_desc)

check_weather: Call this tool to interact with the check_weather API.
What is the check_weather API useful for? Return the weather forecast for the specified location.
Parameters:
- `location` (string, required): The location to check the weather for.

multiply: Call this tool to interact with the multiply API.
What is the multiply API useful for? Multiply two numbers.
Parameters:
- `a` (number, required): The first number.
- `b` (number, required): The second number.

list_files: Call this tool to interact with the list_files API.
What is the list_files API useful for? List all files in the specified directory as a tree.
Parameters:
- `directory` (string, optional): Directory path to list files from. Leave empty to use the default allowed path.

allowed_location_dir: Call this tool to interact with the allowed_location_dir API.
What is the allowed_location_dir API useful for? Return the absolute path of a predefined directory that the agent can access.
Parameters:
- This tool takes no

In [43]:
from prompts import PROMPT_REACT


print(PROMPT_REACT.format(
        tool_descs="tool_descs",
        tool_names=".join([t.name for t in tools])",
        query="query.content"
    ))

Answer the following questions as best you can. You have access to the following tools:

tool_descs

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [.join([t.name for t in tools])]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can be repeated zero or more times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: query.content
Thought: 
